In [ ]:
pip install open3d

In [ ]:
!nvidia-smi

In [ ]:
import numpy as np
import open3d as o3d

In [ ]:
! pip install plotly -q

In [ ]:
!git clone https://github.com/openai/point-e

In [ ]:
%cd /content/point-e

In [ ]:
! pip install -e . -q

In [ ]:
from PIL import Image
import torch
from tqdm.auto import tqdm

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base40M'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.eval()
base_diffusion = diffusion_from_config(DIFFUSION_CONFIGS[base_name])

print('creating upsample model...')
upsampler_model = model_from_config(MODEL_CONFIGS['upsample'], device)
upsampler_model.eval()
upsampler_diffusion = diffusion_from_config(DIFFUSION_CONFIGS['upsample'])

print('downloading base checkpoint...')
base_model.load_state_dict(load_checkpoint(base_name, device))

print('downloading upsampler checkpoint...')
upsampler_model.load_state_dict(load_checkpoint('upsample', device))

In [ ]:
sampler = PointCloudSampler(
    device=device,
    models=[base_model, upsampler_model],
    diffusions=[base_diffusion, upsampler_diffusion],
    num_points=[1024, 4096 - 1024],
    aux_channels=['R', 'G', 'B'],
    guidance_scale=[3.0, 3.0],
)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load an image to condition on.
img = Image.open('Image_file_name')

# Produce a sample from the model.
samples = None
for x in tqdm(sampler.sample_batch_progressive(batch_size=1, model_kwargs=dict(images=[img]))):
    samples = x

In [ ]:
img

In [ ]:
pc = sampler.output_to_point_clouds(samples)[0]

fig = plot_point_cloud(pc, grid_size=3, fixed_bounds=((-0.75, -0.75, -0.75),(0.75, 0.75, 0.75)))

In [ ]:
import plotly.graph_objects as go
fig_plotly = go.Figure(
        data=[
            go.Scatter3d(
                x=pc.coords[:,0], y=pc.coords[:,1], z=pc.coords[:,2],
                mode='markers',
                marker=dict(
                  size=2,
                  color=['rgb({},{},{})'.format(r,g,b) for r,g,b in zip(pc.channels["R"], pc.channels["G"], pc.channels["B"])],
              )
            )
        ],
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        ),
    )

fig_plotly.show(renderer="colab")


In [ ]:
from point_e.util.pc_to_mesh import marching_cubes_mesh

In [ ]:
import torch
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating SDF model....')

name = 'sdf'
model = model_from_config(MODEL_CONFIGS[name], device)
model.eval()

print('loading SDF model....')

model.load_state_dict(load_checkpoint(name, device))

In [ ]:
import skimage.measure as measure

In [ ]:
mesh = marching_cubes_mesh(
    pc = pc,
    model = model,
    batch_size = 4096,
    grid_size = 32,
    progress = True,
)

In [ ]:
with open('starbucks.ply', 'wb') as f:
  mesh.write_ply(f)

#download this ply file

In [ ]:
pip install wget

In [ ]:
import wget
import open3d as o3d


In [ ]:
import open3d as o3d
import numpy as np
import PIL.Image
import IPython.display
import os
import urllib
import tarfile
import gzip
import zipfile
import shutil

In [ ]:
#read_point_cloud reads a point cloud from a file. It tries to decode the file based on the extension name.
pcd = o3d.io.read_point_cloud("starbucks.ply")
print(pcd)
print(np.asarray(pcd.points))

#draw_geometries visualizes the point cloud.
#o3d.visualization.draw_geometries([pcd],width=1920, height=1080, left=50, top=50)

In [ ]:
pcd = pcd.voxel_down_sample(voxel_size=0.0001)  # Adjust voxel_size as needed
print(np.asarray(pcd.points))

In [ ]:
pwd